## Create the raw layer for the US Climate warehouse

### Load data files into BQ tables

#### Create BQ dataset for storing the raw data

In [ ]:
from google.cloud import bigquery

project_id = "kiaraerica"
dataset = "us_climate_raw"
region = "us-central1"

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset kiaraerica.us_climate_raw


#### Common functions

In [1]:
from google.cloud import bigquery

project_id = "kiaraerica"
bucket = "us_climate"
parent_folder = "initial-loads"
region = "us-central1"
dataset = "us_climate_raw"

bq_client = bigquery.Client()

def create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter=",", quote_character="\""):

  uri = f"gs://{bucket}/{parent_folder}/{folder}/{file_name}"
  table_id = f"{project_id}.{dataset}.{table}"

  table = bigquery.Table(table_id, schema=schema)
  table = bq_client.create_table(table, exists_ok=True)
  print("Created table {}".format(table.table_id))

  # remove the data_source and load_time fields before loading the data,
  # neither one is present in the csv
  del schema[-1]
  del schema[-1]
  print(schema)

  job_config = bigquery.LoadJobConfig(
        schema=schema,
        skip_leading_rows=skip_leading_rows,
        source_format=bigquery.SourceFormat.CSV,
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
        field_delimiter=delimiter,
        quote_character=quote_character,
        allow_jagged_rows=True,
        ignore_unknown_values=True
      )

  load_job = bq_client.load_table_from_uri(uri, table_id, job_config=job_config)
  load_job.result()

  destination_table = bq_client.get_table(table_id)
  print("Loaded {} rows.".format(destination_table.num_rows))


def create_load_table_from_json(folder, file_name, table, schema):

  table_id = f"{project_id}.{dataset}.{table}"

  table = bigquery.Table(table_id, schema=schema)
  table = bq_client.create_table(table, exists_ok=True)
  print("Created table {}".format(table.table_id))

  # remove the data_source and load_time fields before loading the data,
  # neither one is present in the json
  del schema[-1]
  del schema[-1]

  #print(schema)

  job_config = bigquery.LoadJobConfig(schema=schema,
      source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
      write_disposition = "WRITE_EMPTY"
  )

  uri = f"gs://{bucket}/{parent_folder}/{folder}/{file_name}"

  load_job = bq_client.load_table_from_uri(
      uri,
      table_id,
      location=region,
      job_config=job_config,
  )

  load_job.result()

  destination_table = bq_client.get_table(table_id)
  print("Loaded {} rows.".format(destination_table.num_rows))

#### Create and load `state_average_temperature`

In [ ]:
folder = "state_temperatures"
file_name = "average_monthly_temperature_by_state_1950-2022.csv"
table = "state_average_temperature"
delimiter = ","
skip_leading_rows = 1

schema = [
  bigquery.SchemaField("month", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("average_temp", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("monthly_mean_from_1901_to_2000", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("centroid_lon", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("centroid_lat", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'kaggle'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table state_average_temperature
[SchemaField('month', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('year', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('state', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('average_temp', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('monthly_mean_from_1901_to_2000', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('centroid_lon', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('centroid_lat', 'FLOAT', 'NULLABLE', None, None, (), None)]
Loaded 41856 rows.


#### Create and load `state_disasters`

In [ ]:
folder = "state_disasters"
file_name = "*.csv"
table = "state_disasters"
delimiter = ","
skip_leading_rows = 3

schema = [
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("drought_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("drought_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("flooding_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("flooding_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("freeze_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("freeze_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("severe_storm_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("severe_storm_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("tropical_cyclone_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("tropical_cyclone_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("wildfire_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("wildfire_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("winter_storm_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("winter_strom_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("all_disasters_count", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("all_disasters_cost_range", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'noaa'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table state_disasters
[SchemaField('state', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('year', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('drought_count', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('drought_cost_range', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('flooding_count', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('flooding_cost_range', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('freeze_count', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('freeze_cost_range', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('severe_storm_count', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('severe_storm_cost_range', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('tropical_cyclone_count', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('tropical_cyclone_cost_range', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('wildfire_count', 'INTEGER'

#### Create and load `facilities_ghg_emissions` and `state_ghg_emissions`

In [ ]:
folder = "epa_ghg_emissions"
file_name = "facilities_ghg.csv"
table = "facility_ghg_emissions"
delimiter = ","
skip_leading_rows = 7

schema = [
  bigquery.SchemaField("facility_id", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("frs_id", "INTEGER", mode="NULLABLE"),
  bigquery.SchemaField("facility_name", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("city", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("naics_code", "INTEGER", mode="NULLABLE"),
  bigquery.SchemaField("year", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("industry_type", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("industry_sector", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("unit_name", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("unit_type", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("unit_reporting_method", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("max_rated_heat_input_capacity", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("carbon_dioxide_emissions", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("methane_emissions", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("nitrous_oxide_emissions", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("biogenic_co2_emissions", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'usepa'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table facility_ghg_emissions
[SchemaField('facility_id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('frs_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('facility_name', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('city', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('state', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('naics_code', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('year', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('industry_type', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('industry_sector', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('unit_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('unit_type', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('unit_reporting_method', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('max_rated_heat_input_capacity', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('carb

In [3]:
folder = "epa_ghg_emissions"
file_name = "state_ghg.csv"
table = "state_ghg_emissions"
delimiter = ","
skip_leading_rows = 1

schema = [
  bigquery.SchemaField("econ_sector", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("econ_subsector", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sector", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("subsector", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("category", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sub_category_1", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sub_category_2", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sub_category_3", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sub_category_4", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("sub_category_5", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("carbon_pool", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("fuel1", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("fuel2", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("geo_ref", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("units", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("ghg", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("ghg_category", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("gwp", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1990", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1991", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1992", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1993", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1994", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1995", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1996", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1997", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1998", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y1999", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2000", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2001", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2002", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2003", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2004", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2005", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2006", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2007", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2008", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2009", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2010", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2011", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2012", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2013", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2014", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2015", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2016", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2017", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2018", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2019", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2020", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2021", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("Y2022", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'usepa'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table state_ghg_emissions
[SchemaField('econ_sector', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('econ_subsector', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sector', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('subsector', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('category', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sub_category_1', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sub_category_2', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sub_category_3', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sub_category_4', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('sub_category_5', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('carbon_pool', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('fuel1', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('fuel2', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('geo_ref', 'STRING', 

#### Create and load `climate_risk_projections`

In [ ]:
folder = "climate_risk_projections"
file_name = "climate_risk.csv"
table = "climate_risk_projections"
delimiter = ","
skip_leading_rows = 1

schema = [
  bigquery.SchemaField("geo_id", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("temp_change", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("precipitation_change", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("extreme_precipitation", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("extreme_cold", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("extreme_heat", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("dry_change", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("impervious_surface", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("housing_density", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("population_estimate", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("low_lying_houses", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("low-lying_roads", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("hazard", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("exposure", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("vulnerability", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("risk", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("risk_percentage", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'nasa'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table climate_risk_projections
[SchemaField('geo_id', 'INTEGER', 'REQUIRED', None, None, (), None), SchemaField('temp_change', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('precipitation_change', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('extreme_precipitation', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('extreme_cold', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('extreme_heat', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('dry_change', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('impervious_surface', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('housing_density', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('population_estimate', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('low_lying_houses', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('low-lying_roads', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('hazard', 'FLOAT', 'REQUIRED', None, None, (), None

#### Create and load `state_climate_policies`

In [ ]:
folder = "state_climate_policies/llm_text"
file_name = "*.jsonl"
table = "state_climate_policies"

schema = [
  bigquery.SchemaField("policy", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("policy_area", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("category", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("status", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("year_enacted", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'climatexchange'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_json(folder, file_name, table, schema)

Created table state_climate_policies
Loaded 3097 rows.


#### Create and load `carbon_capture_facilities`

In [ ]:
folder = "ccus_facility/llm_text"
file_name = "*.jsonl"
table = "carbon_capture_facilities"

schema = [
  bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("facility", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("organization", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("city", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("category", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("status", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("industry", "STRING", mode="NULLABLE"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'bbi-international'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_json(folder, file_name, table, schema)

Created table carbon_capture_facilities
Loaded 135 rows.


#### Create and load `state_electricity_generation_fuel_shares`

In [ ]:
folder = "state_electricity_generation_fuel_shares"
file_name = "state_electricity_generation_fuel_shares.csv"
table = "state_electricity_generation_fuel_shares"
delimiter = ","
skip_leading_rows = 1

schema = [
  bigquery.SchemaField("state", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("nuclear", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("coal", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("natural_gas", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("petroleum", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("hydro", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("geothermal", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("solar_power", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("wind", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("biomass_and_others", "FLOAT", mode="REQUIRED"),
  bigquery.SchemaField("others", "FLOAT", mode="NULLABLE"),
  bigquery.SchemaField("_data_source", "STRING", mode="REQUIRED", default_value_expression="'nei'"),
  bigquery.SchemaField("_load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

create_load_table_from_csv(folder, file_name, table, schema, skip_leading_rows, delimiter)

Created table state_electricity_generation_fuel_shares
[SchemaField('state', 'STRING', 'REQUIRED', None, None, (), None), SchemaField('nuclear', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('coal', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('natural_gas', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('petroleum', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('hydro', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('geothermal', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('solar_power', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('wind', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('biomass_and_others', 'FLOAT', 'REQUIRED', None, None, (), None), SchemaField('others', 'FLOAT', 'NULLABLE', None, None, (), None)]
Loaded 51 rows.
